<p style="font-size:30px; text-align:center; line-height:120%">
    <br> 
        <b>
        COMS 4995 Applied ML
            Homework 4 
        <br></br>
            Wine Reviews: Task 2
        <br></br>
        </b> 
    <br> 
</p>
<p style="font-size:18px; text-align:left; line-height:120%">
    <br> 
        <b>
        Name: Kirit Dhillon, Sagar Lal
        </b>
    <br> 
        <b>
        Uni: ksd2142, sl3946
        </b>
</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loanding and Exploration

In [2]:
data = pd.read_csv("winemag-data-130k-v2.csv")

In [3]:
from sklearn.model_selection import train_test_split

#Split before deciding between text and non-text data to allow for joining later
X_trainval, X_test, y_trainval, y_test = train_test_split(data['description'], data['points'], stratify= data['points'])

In [5]:
print(X_trainval.shape, y_trainval.shape)
print(X_test.shape, y_test.shape)

(97478,) (97478,)
(32493,) (32493,)


### Setting up BOW

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vect = CountVectorizer(stop_words="english", max_features=1000)

X_trainval_bow = bow_vect.fit_transform(X_trainval)
X_test_bow = bow_vect.transform(X_test)

print(X_trainval_bow.shape)
print(X_test_bow.shape)

(97478, 1000)
(32493, 1000)


## Create Word Vector Dataset

In [11]:
import gensim.models.keyedvectors as word2vec
import gensim.models as g

In [12]:
model = g.Doc2Vec.load('./apnews_dbow/doc2vec.bin')

In [16]:
#testing
model.infer_vector(['cat','is','bad']).shape

(300,)

In [34]:
#Create Doc2Vec
list_x_test= list(X_test.str.split(" ", expand = False))
len(list_x_test)

32493

In [41]:
%%time
w2v_x_test = []
j=0
for i in list_x_test:
    j+=1
    if j%10000 == 0:
        print(j)
    w2v_x_test.append(model.infer_vector(i))

10000
20000
30000
CPU times: user 1min 7s, sys: 1.04 s, total: 1min 8s
Wall time: 1min 16s


In [63]:
#Test list to dataframe
w2v_x_test = pd.DataFrame(w2v_x_test)

In [43]:
list_x_trainval= list(X_trainval.str.split(" ", expand = False))
len(list_x_trainval)

97478

In [44]:
%%time
w2v_x_trainval = []
j=0
for i in list_x_trainval:
    j+=1
    if j%10000 == 0:
        print(j)
    w2v_x_trainval.append(model.infer_vector(i))

10000
20000
30000
40000
50000
60000
70000
80000
90000
CPU times: user 3min 13s, sys: 2.74 s, total: 3min 16s
Wall time: 3min 32s


In [46]:
w2v_x_trainval = pd.DataFrame(w2v_x_trainval)
w2v_x_trainval.shape

(97478, 300)

### Run Doc2Vec Model

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

In [64]:
%%time
ridge_pipeline = Pipeline(steps=[
                                ('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, .1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(w2v_x_trainval, y_trainval)

print(("Doc2Vec Ridge score with GridSearchCV %.2f"
       %grid.score(w2v_x_test, y_test)))
print("Best param:", grid.best_params_)

Doc2Vec Ridge score with GridSearchCV 0.45
Best param: {'regressor__alpha': 10}
CPU times: user 12.9 s, sys: 3.99 s, total: 16.9 s
Wall time: 11.7 s


In [52]:
%%time
xgb_pipeline = Pipeline(steps=[
                                ('regressor', XGBRegressor())])

param_grid =  {
               "regressor__max_depth": [4], 
                "regressor__alpha": [0], 
                "regressor__lambda": [0.5],
              }
grid = GridSearchCV(xgb_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(w2v_x_trainval, y_trainval)

print(("XGBoost with GridSearchCV %.2f"
       %grid.score(w2v_x_test, y_test)))
print("Best param:", grid.best_params_)

KeyboardInterrupt: 

### Combine Doc2Vec with BOW

In [61]:
X_trainval_bow = pd.DataFrame(X_trainval_bow.toarray())
X_test_bow = pd.DataFrame(X_test_bow.toarray())

In [67]:
full_X_trainval = pd.concat([X_trainval_bow, w2v_x_trainval], axis=1)
full_X_test = pd.concat([X_test_bow, w2v_x_test],axis=1)

In [68]:
%%time
ridge_pipeline = Pipeline(steps=[
                                ('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, 0.1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(full_X_trainval, y_trainval)

print(("Baseline Ridge score with GridSearchCV %.2f"
       %grid.score(full_X_test, y_test)))
print("Best param:", grid.best_params_)

Baseline Ridge score with GridSearchCV 0.64
Best param: {'regressor__alpha': 10}
CPU times: user 2min 2s, sys: 37.7 s, total: 2min 40s
Wall time: 2min 2s
